In [2]:
!pip install --upgrade torch
!pip install -U datasets fsspec huggingface_hub
!pip install --upgrade vllm
!pip install latex2sympy2-extended
!pip install 'math-verify[inference]'

  Using cached torch-2.7.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.7.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cache

In [6]:
# from datasets import load_dataset

# ds = load_dataset("openai/gsm8k", "main")
# #dataset = load_dataset('squad', split='train')


#ds = load_dataset("openai/gsm8k", "socratic")

In [4]:
len(ds["train"]["question"])
len(ds["train"]["answer"])

7473

In [11]:
# print(ds["train"]["answer"][0])
# print(ds["train"]["answer"][0].split('\n#### ')[1])




# def process_ground_truth(ans):
#     return ans.split('\n#### ')[1]

# with ProcessPoolExecutor() as executor:
#     ground_truth = list(executor.map(process_ground_truth, ds["train"]["answer"]))
# len(ground_truth)



Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72
72


7473

In [3]:
from datasets import load_dataset
from vllm import LLM, SamplingParams
from drgrpo_grader import r1_zero_reward_fn
from concurrent.futures import ThreadPoolExecutor
import os
import json
def preprocess_dataset(ds):
    questions, answers =  ds["train"]["question"], ds["train"]["answer"]
    with open("prompts/r1_zero.prompt", "r", encoding="utf-8") as f:
        prompt_string = f.read()
    def process_question(q):
        return prompt_string.format(question=q)
    def process_ground_truth(ans):
        return ans.split('\n#### ')[1]

    with ThreadPoolExecutor() as executor:
        question_prompts = list(executor.map(process_question, ds["train"]["question"]))
    with ThreadPoolExecutor() as executor:
        ground_truth = list(executor.map(process_ground_truth, ds["train"]["answer"]))
    return question_prompts, ground_truth

def generate(model="Qwen/Qwen2.5-1.5B"):
    llm = LLM(model, dtype="float16")
    # Create a sampling params object, stopping generation on newline.
    sampling_params = SamplingParams(
    temperature=1.0, top_p=1.0, max_tokens=1024, stop=["</answer>"]
    )
    sampling_params.include_stop_str_in_output = True
    
    # Generate texts from the prompts. The output is a list of RequestOutput objects
    # that contain the prompt, generated text, and other information.
    model_outputs = llm.generate(question_prompts, sampling_params)
    return model_outputs

def evaluate(model_outputs, ground_truth, out_dir ='./eval/baseline_eval.jsonl'):
    # Print the outputs.
    with open(out_dir, "a", encoding="utf-8") as f:
        format_reward, answer_reward, reward = 0, 0, 0
        for output, gt in zip(model_outputs, ground_truth):
            prompt = output.prompt
            generated_text = output.outputs[0].text
            
            res = r1_zero_reward_fn(generated_text, gt)
            format_reward+= res['format_reward']
            answer_reward+= res['answer_reward']
            reward+= res['reward']
            print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}, format_reward: {str(res['format_reward'])}, answer_reward: {str(res['answer_reward'])}, reward: {str(res['reward'])}")
            #final_output.append([res['format_reward'], res['answer_reward'], res['reward']])
            dp = {
                "prompt": f"{prompt}",
                "output": f"{generated_text}",
                "format_reward": res['format_reward'],
                "answer_reward": res['answer_reward'],
                "reward": res['reward']
            }
              
            json.dump(dp, f, ensure_ascii=False)
            f.write("\n")
    return  format_reward, answer_reward, reward
    

ds = load_dataset("openai/gsm8k", "main")

question_prompts, ground_truth = preprocess_dataset(ds)
model_outputs = generate(model="Qwen/Qwen2.5-1.5B")
evaluate(model_outputs, ground_truth, out_dir ='./eval/baseline_eval.jsonl')








WARNING 06-03 07:41:58 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 06-03 07:41:58 config.py:542] This model supports multiple tasks: {'embed', 'generate', 'score', 'classify', 'reward'}. Defaulting to 'generate'.
WARNING 06-03 07:41:58 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 06-03 07:41:58 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 06-03 07:41:58 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='Qwen/Qwen2.5-1.5B', speculative_config=None, tokenizer='Qwen/Qwen2.5-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=131072, download_dir=None, load_

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

RuntimeError: Data processing error: CAS service error : IO Error: No space left on device (os error 28)

In [5]:

#r1_zero_reward_fn(response, ground_truth, fast=True)
# Sample prompts.
# prompts = [
# "Hello, my name is",
# "The president of the United States is",
# "The capital of France is",
# "The future of AI is",
# ]
# Create a sampling params object, stopping generation on newline.
sampling_params = SamplingParams(
temperature=1.0, top_p=1.0, max_tokens=1024, stop=["</answer>"]
)
sampling_params.include_stop_str_in_output = True
# Create an LLM.

llm = LLM(model="Qwen/Qwen2.5-1.5B")

# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(ds["train"]["question"], sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

INFO 06-02 05:22:20 [__init__.py:247] No platform detected, vLLM is running on UnspecifiedPlatform


2025-06-02 05:22:21.462756: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


WARNING 06-02 05:22:26 [_custom_ops.py:21] Failed to import from vllm._C with ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')
INFO 06-02 05:22:28 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-02 05:22:28 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-02 05:22:28 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.


RuntimeError: Device string must not be empty

In [4]:
llm = LLM(model="Qwen/Qwen2.5-1.5B", dtype="float16")

WARNING 06-03 07:47:22 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 06-03 07:47:22 config.py:542] This model supports multiple tasks: {'embed', 'generate', 'score', 'classify', 'reward'}. Defaulting to 'generate'.
WARNING 06-03 07:47:22 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 06-03 07:47:22 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 06-03 07:47:22 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='Qwen/Qwen2.5-1.5B', speculative_config=None, tokenizer='Qwen/Qwen2.5-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=131072, download_dir=None, load_

/home/sagemaker-user/.conda/envs/myenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:799: UserWarning: Not enough free disk space to download the file. The expected file size is: 3087.47 MB. The target location /home/sagemaker-user/.cache/huggingface/hub/models--Qwen--Qwen2.5-1.5B/blobs only has 1.67 MB free disk space.
  warnings.warn(


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

RuntimeError: Data processing error: CAS service error : IO Error: No space left on device (os error 28)